In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
import warnings
import pandas as pd

warnings.simplefilter(action='ignore')
data_patient=pd.read_csv("patient_info.csv")
data_environment=pd.read_csv("environment.csv")
data_questionnaire=pd.read_csv("questionnaire.csv")
data=pd.merge(data_environment, data_questionnaire, on=['user_key','date'])
data_final=pd.merge(data, data_patient, on=['user_key'])
data_final.columns
data_final.interpolate(method="linear")

data_final['grass_pollen']=data_final['grass_pollen'].replace('Low',1)
data_final['grass_pollen']=data_final['grass_pollen'].replace('Moderate',2)
data_final['grass_pollen']=data_final['grass_pollen'].replace('High',3)
data_final['tree_pollen']=data_final['tree_pollen'].replace('Low',1)
data_final['tree_pollen']=data_final['tree_pollen'].replace('Moderate',2)
data_final['tree_pollen']=data_final['tree_pollen'].replace('High',3)
data_final['weed_pollen']=data_final['weed_pollen'].replace('Low',1)
data_final['weed_pollen']=data_final['weed_pollen'].replace('Moderate',2)
data_final['weed_pollen']=data_final['weed_pollen'].replace('High',3)
data_final['weed_pollen']=data_final['weed_pollen'].replace('Very High',4)
data_final['daily_night_symp']=data_final['daily_night_symp'].replace(True,2)
data_final['daily_night_symp']=data_final['daily_night_symp'].replace(False,1)
data_final['daily_day_symp']=data_final['daily_day_symp'].replace(True,2)
data_final['daily_day_symp']=data_final['daily_day_symp'].replace(False,1)
data_final['daily_limit_activity']=data_final['daily_limit_activity'].replace(True,2)
data_final['daily_limit_activity']=data_final['daily_limit_activity'].replace(False,1)
data_final['sex']=data_final['sex'].replace('male',1)
data_final['sex']=data_final['sex'].replace('female',2)
data_final['age_range']=data_final['age_range'].replace('18-29yo',1)
data_final['age_range']=data_final['age_range'].replace('30-39yo',2)
data_final['age_range']=data_final['age_range'].replace('40-49yo',3)
data_final['age_range']=data_final['age_range'].replace('50+yo',4)

data_final['bmi_range']=data_final['bmi_range'].replace('Normal',1)
data_final['bmi_range']=data_final['bmi_range'].replace('Obesity',3)
data_final['bmi_range']=data_final['bmi_range'].replace('Pre-obesity',2)

data_final['age_diagnosed_range']=data_final['age_diagnosed_range'].replace('0-6yo',1)
data_final['age_diagnosed_range']=data_final['age_diagnosed_range'].replace('7-11yo',2)
data_final['age_diagnosed_range']=data_final['age_diagnosed_range'].replace('12-18yo',3)
data_final['age_diagnosed_range']=data_final['age_diagnosed_range'].replace('19+yo',4)

# Never (<100 cigarettes in lifetime)
data_final['smoker']=data_final['smoker'].replace('Never (<100 cigarettes in lifetime)',1)
data_final['smoker']=data_final['smoker'].replace('Previous',2)
# Mixed/multiple ethnic groups
data_final['race']=data_final['race'].replace('Mixed/multiple ethnic groups',1)
data_final['race']=data_final['race'].replace('White',2)
# Severe
data_final['severity']=data_final['severity'].replace('Mild',1)
data_final['severity']=data_final['severity'].replace('Moderate',2)
data_final['severity']=data_final['severity'].replace('Severe',3)
data_final['severity']=data_final['severity'].replace('Very severe',4)

data_final.fillna(-1, inplace=True)
# data_final.to_excel("result.xlsx")
data_final['grass_pollen']


0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
1495    2.0
1496    2.0
1497    2.0
1498    1.0
1499    1.0
Name: grass_pollen, Length: 1500, dtype: float64

In [3]:
import statsmodels.api as sm
import pandas as pd
data_result=pd.read_csv("result.csv")


factors_independent=['temperature',
'temperature_min',
'temperature_max',
'pressure',
'humidity',
'wind_speed',
'wind_deg',
'aqi',
'co',
'no',
'no2',
'o3',
'so2',
'pm2_5',
'pm10',
'nh3',
'grass_pollen',
'tree_pollen',
'weed_pollen',
't1',
't2',
't3',
't4',
't5',
't6',
't7',
't8',
't9',
't10',
't11',
't12',
't13',
't14',
't15',
't16',
't17',
't18',
't19',
't20',
'sex',
'age_range',
'bmi_range',
'age_diagnosed_range',
'max_pef_expected',
'smoker',
'pack_years',
'race',
'severity']

data_result=data_result.dropna()

dependent=data_result[['daily_night_symp',
'daily_day_symp']]

night_dependent=data_result['daily_night_symp']
day_dependent=data_result['daily_day_symp']

# Method used to perform forward regression, freshly copied no modification from previous assignment
def forward_regression_method(X, y,
                   
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    initial_list = []
    selected = list(initial_list)
    bicresult={}
    print(selected)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(selected))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[selected+[new_column]]))).fit()
          
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            selected.append(best_feature)
            changed=True
            if verbose:
                print('Adding {:15}  with the p-value {:.6} '.format(best_feature, best_pval))
        else:
            bad_feature=new_pval.idxmin()
            print('Removing {:15}  with the p-value {:.6} '.format(bad_feature, best_pval))
        if not changed:
            break

    return selected

data_result['t1']=data_result['t1'].replace(True,2)
data_result['t1']=data_result['t1'].replace(False,1)
data_result['t2']=data_result['t2'].replace(True,2)
data_result['t2']=data_result['t2'].replace(False,1)
data_result['t3']=data_result['t3'].replace(True,2)
data_result['t3']=data_result['t3'].replace(False,1)
data_result['t4']=data_result['t4'].replace(True,2)
data_result['t4']=data_result['t4'].replace(False,1)
data_result['t5']=data_result['t5'].replace(True,2)
data_result['t5']=data_result['t5'].replace(False,1)
data_result['t6']=data_result['t6'].replace(True,2)
data_result['t6']=data_result['t6'].replace(False,1)
data_result['t7']=data_result['t7'].replace(True,2)
data_result['t7']=data_result['t7'].replace(False,1)
data_result['t8']=data_result['t8'].replace(True,2)
data_result['t8']=data_result['t8'].replace(False,1)
data_result['t9']=data_result['t9'].replace(True,2)
data_result['t9']=data_result['t9'].replace(False,1)
data_result['t10']=data_result['t10'].replace(True,2)
data_result['t10']=data_result['t10'].replace(False,1)
data_result['t11']=data_result['t11'].replace(True,2)
data_result['t11']=data_result['t11'].replace(False,1)
data_result['t12']=data_result['t12'].replace(True,2)
data_result['t12']=data_result['t12'].replace(False,1)
data_result['t13']=data_result['t13'].replace(True,2)
data_result['t13']=data_result['t13'].replace(False,1)
data_result['t14']=data_result['t14'].replace(True,2)
data_result['t14']=data_result['t14'].replace(False,1)
data_result['t15']=data_result['t15'].replace(True,2)
data_result['t15']=data_result['t15'].replace(False,1)
data_result['t16']=data_result['t16'].replace(True,2)
data_result['t16']=data_result['t16'].replace(False,1)
data_result['t17']=data_result['t17'].replace(True,2)
data_result['t17']=data_result['t17'].replace(False,1)
data_result['t18']=data_result['t18'].replace(True,2)
data_result['t18']=data_result['t18'].replace(False,1)
data_result['t19']=data_result['t19'].replace(True,2)
data_result['t19']=data_result['t19'].replace(False,1)
data_result['t20']=data_result['t20'].replace(True,2)
data_result['t20']=data_result['t20'].replace(False,1)



independent=data_result[factors_independent]

selected_night=forward_regression_method(independent,night_dependent)
print("selected  for night are:")
print(selected_night)

selected_day=forward_regression_method(independent,day_dependent)
print("selected  for day are:")
print(selected_day)

[]
Adding t20              with the p-value 1.43495e-41 
Adding max_pef_expected  with the p-value 5.25313e-29 
Adding bmi_range        with the p-value 4.93327e-16 
Adding t19              with the p-value 5.14059e-14 
Adding t1               with the p-value 7.95817e-07 
Adding smoker           with the p-value 1.25828e-11 
Adding o3               with the p-value 1.23836e-06 
Adding age_range        with the p-value 4.3548e-05 
Adding sex              with the p-value 0.000666271 
Adding t2               with the p-value 0.000708901 
Adding t12              with the p-value 0.00731697 
Adding t5               with the p-value 0.00514604 
Adding t16              with the p-value 0.0327353 
Adding pressure         with the p-value 0.0374171 
Removing wind_speed       with the p-value 0.0583169 
selected  for night are:
['t20', 'max_pef_expected', 'bmi_range', 't19', 't1', 'smoker', 'o3', 'age_range', 'sex', 't2', 't12', 't5', 't16', 'pressure']
[]
Adding t1               with the p-va

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

independent = data_result[selected_day]
outcome_day = day_dependent
outcome_night = night_dependent

day_x_train, day_x_test, day_y_train, day_y_test = train_test_split(independent, day_dependent, test_size=0.2)
model_nb_day = GaussianNB()
model_nb_day.fit(day_x_train, day_y_train)
y_pred_day = model_nb_day.predict(day_x_test)
accuracy_day = accuracy_score(day_y_test, y_pred_day)
print(accuracy_day)

independent = data_result[selected_night]
night_x_train, night_x_test, night_y_train, night_y_test = train_test_split(independent, night_dependent, test_size=0.2)
model_nb_night = GaussianNB()
model_nb_night.fit(night_x_train, night_y_train)
y_pred_night = model_nb_night.predict(night_x_test)
accuracy_night = accuracy_score(night_y_test, y_pred_night)
print(accuracy_night)

# using f1 score
f1_night = f1_score(night_y_test, y_pred_night)
f1_day = f1_score(day_y_test, y_pred_day)
print("F1 Score for night:", f1_night)
print("F1 Score for day:", f1_day)

0.85
0.71
F1 Score for night: 0.779746835443038
F1 Score for day: 0.6456692913385828


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

independent = data_result[selected_day]
outcome_day = day_dependent
outcome_night = night_dependent

day_x_train, day_x_test, day_y_train, day_y_test = train_test_split(independent, day_dependent, test_size=0.2)
model_svm_day = SVC(kernel='rbf', gamma='auto')  # You can adjust the parameters as needed
model_svm_day.fit(day_x_train, day_y_train)
y_pred_day = model_svm_day.predict(day_x_test)
accuracy_day = accuracy_score(day_y_test, y_pred_day)
#print(accuracy_day)

independent = data_result[selected_night]
night_x_train, night_x_test, night_y_train, night_y_test = train_test_split(independent, night_dependent, test_size=0.2)
model_svm_night = SVC(kernel='rbf', gamma='auto')  # You can adjust the parameters as needed
model_svm_night.fit(night_x_train, night_y_train)
y_pred_night = model_svm_night.predict(night_x_test)
accuracy_night = accuracy_score(night_y_test, y_pred_night)
#print(accuracy_night)

# using f1 score
f1_night = f1_score(night_y_test, y_pred_night)
f1_day = f1_score(day_y_test, y_pred_day)

print("Support vector machine result:")

print("Accuracy Score for night:", accuracy_night)
print("Accuracy Score for day:", accuracy_day)
print("F1 Score for night:", f1_night)
print("F1 Score for day:", f1_day)

Support vector machine result:
Accuracy Score for night: 0.8033333333333333
Accuracy Score for day: 0.86
F1 Score for night: 0.8858800773694391
F1 Score for day: 0.6666666666666667


In [6]:
from tabulate import tabulate

# Assuming you have computed the accuracy and F1 scores and stored them in variables accuracy_night, accuracy_day, f1_night, and f1_day

# Data to be tabulated
data = [
    ["Accuracy Score for night:", accuracy_night],
    ["Accuracy Score for day:", accuracy_day],
    ["F1 Score for night:", f1_night],
    ["F1 Score for day:", f1_day]
]

# Tabulate the data
table = tabulate(data, headers=["Metric", "Score"], tablefmt="grid")

# Print the tabulated results
print("Support vector machine result:")
print(table)

Support vector machine result:
+---------------------------+----------+
| Metric                    |    Score |
+===========================+==========+
| Accuracy Score for night: | 0.803333 |
+---------------------------+----------+
| Accuracy Score for day:   | 0.86     |
+---------------------------+----------+
| F1 Score for night:       | 0.88588  |
+---------------------------+----------+
| F1 Score for day:         | 0.666667 |
+---------------------------+----------+


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

independent = data_result[selected_day]
outcome_day = day_dependent
outcome_night = night_dependent

day_x_train, day_x_test, day_y_train, day_y_test = train_test_split(independent, day_dependent, test_size=0.2)
model_rf_day = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the parameters as needed
model_rf_day.fit(day_x_train, day_y_train)
y_pred_day = model_rf_day.predict(day_x_test)
accuracy_day = accuracy_score(day_y_test, y_pred_day)
print(accuracy_day)

independent = data_result[selected_night]
night_x_train, night_x_test, night_y_train, night_y_test = train_test_split(independent, night_dependent, test_size=0.2)
model_rf_night = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the parameters as needed
model_rf_night.fit(night_x_train, night_y_train)
y_pred_night = model_rf_night.predict(night_x_test)
accuracy_night = accuracy_score(night_y_test, y_pred_night)
print(accuracy_night)

# using f1 score
f1_night = f1_score(night_y_test, y_pred_night)
f1_day = f1_score(day_y_test, y_pred_day)
print("F1 Score for night:", f1_night)
print("F1 Score for day:", f1_day)

0.9
0.8466666666666667
F1 Score for night: 0.9083665338645419
F1 Score for day: 0.7761194029850746


## OTHER TESTED MODELS

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from tabulate import tabulate

# Assuming data_result, selected_day, selected_night, day_dependent, and night_dependent are defined elsewhere

# For day classification
day_x_train, day_x_test, day_y_train, day_y_test = train_test_split(data_result[selected_day], day_dependent, test_size=0.2)
model_knn_day = KNeighborsClassifier()  # You can adjust the parameters as needed
model_knn_day.fit(day_x_train, day_y_train)
y_pred_day = model_knn_day.predict(day_x_test.values)  # Ensure day_x_test is in the correct format
accuracy_day = accuracy_score(day_y_test, y_pred_day)
f1_day = f1_score(day_y_test, y_pred_day)

# For night classification
night_x_train, night_x_test, night_y_train, night_y_test = train_test_split(data_result[selected_night], night_dependent, test_size=0.2)
model_knn_night = KNeighborsClassifier()  # You can adjust the parameters as needed
model_knn_night.fit(night_x_train, night_y_train)
y_pred_night = model_knn_night.predict(night_x_test.values)  # Ensure night_x_test is in the correct format
accuracy_night = accuracy_score(night_y_test, y_pred_night)
f1_night = f1_score(night_y_test, y_pred_night)

# Tabulate results
print("KNearest Neighbours result:")
data = [
    ["Accuracy Score", accuracy_night, accuracy_day],
    ["F1 Score", f1_night, f1_day]
]
table = tabulate(data, headers=["Metric", "Night", "Day"], tablefmt="grid")
print(table)



KNearest Neighbours result:
+----------------+----------+----------+
| Metric         |    Night |      Day |
+================+==========+==========+
| Accuracy Score | 0.803333 | 0.85     |
+----------------+----------+----------+
| F1 Score       | 0.878351 | 0.645669 |
+----------------+----------+----------+


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from tabulate import tabulate

# Assuming data_result, selected_day, selected_night, day_dependent, and night_dependent are defined elsewhere

# For day classification
day_x_train, day_x_test, day_y_train, day_y_test = train_test_split(data_result[selected_day], day_dependent, test_size=0.2)
model_logreg_day = LogisticRegression()  # You can adjust the parameters as needed
model_logreg_day.fit(day_x_train, day_y_train)
y_pred_day = model_logreg_day.predict(day_x_test)  # Ensure day_x_test is in the correct format
accuracy_day = accuracy_score(day_y_test, y_pred_day)
f1_day = f1_score(day_y_test, y_pred_day)

# For night classification
night_x_train, night_x_test, night_y_train, night_y_test = train_test_split(data_result[selected_night], night_dependent, test_size=0.2)
model_logreg_night = LogisticRegression()  # You can adjust the parameters as needed
model_logreg_night.fit(night_x_train, night_y_train)
y_pred_night = model_logreg_night.predict(night_x_test)  # Ensure night_x_test is in the correct format
accuracy_night = accuracy_score(night_y_test, y_pred_night)
f1_night = f1_score(night_y_test, y_pred_night)

# Tabulate results
print("Logistic Regression result:")
data = [
    ["Accuracy Score", accuracy_night, accuracy_day],
    ["F1 Score", f1_night, f1_day]
]
table = tabulate(data, headers=["Metric", "Night", "Day"], tablefmt="grid")
print(table)


Logistic Regression result:
+----------------+----------+----------+
| Metric         |    Night |      Day |
+================+==========+==========+
| Accuracy Score | 0.886667 | 0.88     |
+----------------+----------+----------+
| F1 Score       | 0.93254  | 0.678571 |
+----------------+----------+----------+


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from tabulate import tabulate

# Assuming data_result, selected_day, selected_night, day_dependent, and night_dependent are defined elsewhere

# For day classification
day_x_train, day_x_test, day_y_train, day_y_test = train_test_split(data_result[selected_day], day_dependent, test_size=0.2)
model_dt_day = DecisionTreeClassifier()  # You can adjust the parameters as needed
model_dt_day.fit(day_x_train, day_y_train)
y_pred_day = model_dt_day.predict(day_x_test)  # Ensure day_x_test is in the correct format
accuracy_day = accuracy_score(day_y_test, y_pred_day)
f1_day = f1_score(day_y_test, y_pred_day)

# For night classification
night_x_train, night_x_test, night_y_train, night_y_test = train_test_split(data_result[selected_night], night_dependent, test_size=0.2)
model_dt_night = DecisionTreeClassifier()  # You can adjust the parameters as needed
model_dt_night.fit(night_x_train, night_y_train)
y_pred_night = model_dt_night.predict(night_x_test)  # Ensure night_x_test is in the correct format
accuracy_night = accuracy_score(night_y_test, y_pred_night)
f1_night = f1_score(night_y_test, y_pred_night)

# Tabulate results
print("Decision Trees result:")
data = [
    ["Accuracy Score", accuracy_night, accuracy_day],
    ["F1 Score", f1_night, f1_day]
]
table = tabulate(data, headers=["Metric", "Night", "Day"], tablefmt="grid")
print(table)


Decision Trees result:
+----------------+----------+----------+
| Metric         |    Night |      Day |
+================+==========+==========+
| Accuracy Score | 0.766667 | 0.863333 |
+----------------+----------+----------+
| F1 Score       | 0.851695 | 0.754491 |
+----------------+----------+----------+
